tempo:
- ammazzare 

- ingannare 

- pioggia 

- mele 

- spazio

Cercare le parole nelle frasi del dataset, restiruire le frasi che le cotnengono.
pulire le frasi ottenute, per restituire "tempo" da "ammazzare il tempo" (stopwords e initialwords)
restituirer la parola più frequente tra quelle restituite

In [72]:
from nltk.corpus import stopwords

In [73]:
WORDS_1 = ['ammazzare', 'ingannare', 'pioggia', 'mele', 'spazio']
WORDS_2 = ['bicchiere', 'mulino', 'gocce', 'gola', 'buco']
WORDS_3 = ['campa', 'battaglia', 'razza', 'Troia', 'vecchio']
WORDS_4 = ['giallo', 'cane', 'scimmia', 'rubato', 'amore']
WORDS_5 = ['attività', 'famiglia', 'rubato', 'fiume', 'nido']
WORDS_6 = ['color', 'dietro', 'pietra', 'bianca', 'andare']
WORDS_7 = ['bollicine', 'da Gama']
WORDS_8 = ['sangue', 'auto', 'vertigo', 'zucchero', 'cielo']

In [74]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

In [75]:
def check_dataset(path, words):
    phrases = []    
    with open(path, 'r') as file:
        for line in file:
            for word in words:
                if word.lower() in line.lower():  # Salviamo le frasi che contengono le parole in input
                    line = line.lower()
                    line = line.replace(word, '')
                    phrases.append(remove_stop_words_ita(line)) # Rimuoviamo dalla frase le stopword e le parole in input
    return phrases

In [76]:
def most_frequent_words(phrases):
    words = {}
    for phrase in phrases:
        for word in phrase:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    return words

In [77]:
def get_most_frequent_word(words):
    max_word = ''
    max_count = 0
    for word in words:
        if words[word] > max_count:
            max_word = word
            max_count = words[word]
    return max_word

In [78]:
ph = check_dataset('..\ghigliottina.txt', WORDS_8)
#print(ph)
dict = most_frequent_words(ph)
print(dict)
answer = get_most_frequent_word(dict)
print(f"\n{WORDS_8}La sesta parola è: {answer}")

{'blu': 5, 'apriti': 1, '!': 1, 'tuttavia': 1, 'risulta': 1, 'documenti': 1, 'storici': 1, 'stato': 1, 'proprio': 1, 'buridano': 1, 're': 1, 'tale': 1, 'apologo': 1, 'cavar': 2, 'rapa': 1, 'muro': 1, 'gelare': 1, 'vene': 1, 'lacrime': 1, 'altra': 1, 'metã': 1, 'settimo': 1, 'essere': 1, 'sxm': 1, 'misto': 1, 'oro,': 1, 'incenso': 1, 'birra': 1, 'fornaciari': 1, 'freddo': 1, 'teatro': 1, 'orrori': 1, 'metallo': 2, 'sopra': 1, 'berlino': 1, '(der': 1, 'himmel': 1, 'ã¼ber': 1, 'berlin),': 1, 'regia': 4, 'wim': 1, 'wenders': 1, '(1987)': 1, 'donna': 1, 'visse': 1, 'due': 1, 'volte': 1, '(),': 1, 'alfred': 1, 'hitchcock': 1, '(1958)': 1, '(aggiunto': 2, '2012)': 2, 'giorni': 1, '(days': 1, 'of': 1, 'heaven),': 1, 'terrence': 1, 'malick': 1, '(1978)': 1, '(kind': 1, 'hearts': 1, 'and': 1, 'coronets),': 1, 'robert': 1, 'hamer': 1, '(1949)': 1}

['sangue', 'auto', 'vertigo', 'zucchero', 'cielo']La sesta parola è: blu
